# Data preperation notebook

In [1]:
import requests
import time
import pandas as pd

# Define the endpoint and parameters
base_url = "https://api-testnet.bybit.com"
endpoint = "/v5/market/kline"
symbol = "BTCUSD"  # Example trading pair
interval = "15"    # Kline interval, e.g., "1", "5", "15", "60", "240", etc.
limit = 10        # Number of klines to fetch
start_time = int(time.time()) - 3600 * 24  # Start time in seconds (e.g., 24 hours ago)

# Construct the URL
url = f"{base_url}{endpoint}?category=linear&symbol={symbol}&interval={interval}&limit={limit}"#&start_time={start_time}"

# Make the GET request
response = requests.get(url,headers={}, data={})
data = response.json() 

print(data)
print(pd.Timestamp(data["time"],unit="ms"))
print(data["result"]["list"])



{'retCode': 0, 'retMsg': 'OK', 'result': {'symbol': 'BTCUSD', 'category': 'linear', 'list': [['1717047000000', '68065', '68070.5', '68052', '68053.5', '367534', '5.39970161'], ['1717046100000', '68006.5', '68075', '67990', '68065', '2317435', '34.06543447'], ['1717045200000', '68120', '68135', '68006.5', '68006.5', '2458651', '36.12209538'], ['1717044300000', '68186', '68186', '68107', '68120', '2779844', '40.79392886'], ['1717043400000', '68197', '68393.5', '68181', '68186', '4760662', '69.70852656'], ['1717042500000', '68002', '68200', '68001', '68197', '4181886', '61.40196159'], ['1717041600000', '68067.5', '68087.5', '67947', '68002', '2924624', '42.99343437'], ['1717040700000', '68011.5', '68140', '68006', '68067.5', '2634776', '38.70310042'], ['1717039800000', '67988', '68024.5', '67966.5', '68011.5', '2526190', '37.14812638'], ['1717038900000', '67904.5', '67988', '67903.5', '67988', '2198972', '32.36367862']]}, 'retExtInfo': {}, 'time': 1717047239466}
2024-05-30 05:33:59.466000

In [2]:
#def prepare_data(data):
print(pd.to_datetime(int(data["result"]["list"][0][0]),unit='ms'))
print(pd.to_datetime(int(data["result"]["list"][1][0]),unit='ms'))
print(pd.to_datetime(int(data["result"]["list"][2][0]),unit='ms'))
print(pd.to_datetime(int(data["result"]["list"][3][0]),unit='ms'))



2024-05-30 05:30:00
2024-05-30 05:15:00
2024-05-30 05:00:00
2024-05-30 04:45:00


In [3]:
import pandas as pd
import numpy as np

def prepare_data(data):
    
    # define pandas DF 
    cols = ['time','open', 'high', 'low', 'close','vol','vol_coin']
    df = pd.DataFrame(data["result"]["list"], columns=cols)
    
    df['time'] = df['time'].astype(int)
    
    df["time"] = pd.to_datetime(df["time"],unit="ms")
    df["day"] = df["time"].dt.day
    df["hour"] = df["time"].dt.hour

    # the 10-day moving average
    df["10MA"] = df["close"].astype('float64').mean()
    
    # Calculate the standard deviation of the closing prices over the same 20-day period
    df["10STD"] = df["close"].astype('float64').std()
    
    # volume weighted close price
    df["vwap"] = df.close.astype('float64')/df.vol.astype('float64')
    
    # Bollinger Band: 
    # simple_moving_average(20) + std x 2 | sma - std x 2
    df["bb_upper_band"] = df["10MA"].astype('float64') + 2 * df["10STD"].astype('float64')
    df["bb_lower_band"] = df["10MA"].astype('float64') - 2 * df["10STD"].astype('float64')

    # Calculate On-Balance Volume (OBV)
    df["change"] = df["close"].astype('float64').diff(periods=-1)#.shift(-1)
    df["direction"] = df["change"].astype('float64').apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    df["obv"] = df["vol"].astype('float64') * df["direction"].astype('float64')
    df["obv"] = df["obv"].astype('float64').cumsum()

    # Stack the DataFrame for all non-constant rows of the period (OHLC,vol,vwap,obv)
    stacked_df_cols = ["open","high","low","close","vol","vol_coin","vwap","obv","change","direction"]
    stacked_df = df.loc[0:len(df)-2,stacked_df_cols].stack() #cut the last row 0:8=9, (10-2) bc of diff = NAN
    
    # Create a new DataFrame from the stacked series and transpose it
    # this creates a multi index data frame with tuples as indices, like [(0,'ts'),...]
    df_single_row = stacked_df.to_frame().T
    
    # now change the multiindex col to a single index col by replacing it witht a list of concatenated strings 
    df_single_row.columns = [f'{col[1]}_{col[0]}' for col in df_single_row.columns]

    # get the first row of the bollinger band metrics that are a constant for the 10 observations
    bb_cols = ["10MA","10STD","bb_upper_band","bb_lower_band"]
    df_bb_metrics = df[bb_cols].head(1)

    # collect meta data
    meta_cols = ["time","day","hour"]
    df_meta = df[meta_cols].head(1)

    # put cols side by side
    df_concat = pd.concat([df_meta,df_bb_metrics,df_single_row],axis=1)
    
    return df,stacked_df,df_single_row,df_concat
    
df,stack_df,single_df,prep_df = prepare_data(data)

In [4]:
df

,time,open,high,low,close,vol,vol_coin,day,hour,10MA,10STD,vwap,bb_upper_band,bb_lower_band,change,direction,obv
0,2024-05-30 05:30:00,68065,68070.5,68052,68053.5,367534,5.39970161,30,5,68069.7,75.417431,0.185162,68220.534862,67918.865138,-11.5,-1,-367534.0
1,2024-05-30 05:15:00,68006.5,68075,67990,68065,2317435,34.06543447,30,5,68069.7,75.417431,0.029371,68220.534862,67918.865138,58.5,1,1949901.0
2,2024-05-30 05:00:00,68120,68135,68006.5,68006.5,2458651,36.12209538,30,5,68069.7,75.417431,0.027660,68220.534862,67918.865138,-113.5,-1,-508750.0
3,2024-05-30 04:45:00,68186,68186,68107,68120,2779844,40.79392886,30,4,68069.7,75.417431,0.024505,68220.534862,67918.865138,-66.0,-1,-3288594.0
4,2024-05-30 04:30:00,68197,68393.5,68181,68186,4760662,69.70852656,30,4,68069.7,75.417431,0.014323,68220.534862,67918.865138,-11.0,-1,-8049256.0
5,2024-05-30 04:15:00,68002,68200,68001,68197,4181886,61.40196159,30,4,68069.7,75.417431,0.016308,68220.534862,67918.865138,195.0,1,-3867370.0
6,2024-05-30 04:00:00,68067.5,68087.5,67947,68002,2924624,42.99343437,30,4,68069.7,75.417431,0.023252,68220.534862,67918.865138,-65.5,-1,-6791994.0
7,2024-05-30 03:45:00,68011.5,68140,68006,68067.5,2634776,38.70310042,30,3,68069.7,75.417431,0.025834,68220.534862,67918.865138,56.0,1,-4157218.0
8,2024-05-30 03:30:00,67988,68024.5,67966.5,68011.5,2526190,37.14812638,30,3,68069.7,75.417431,0.026923,68220.534862,67918.865138,23.5,1,-1631028.0
9,2024-05-30 03:15:00,67904.5,67988,67903.5,67988,2198972,32.36367862,30,3,68069.7,75.417431,0.030918,68220.534862,67918.865138,NaN,0,-1631028.0


In [5]:
stack_df[8]

open               67988
high             68024.5
low              67966.5
close            68011.5
vol              2526190
vol_coin     37.14812638
vwap            0.026923
obv           -1631028.0
change              23.5
direction              1
dtype: object

In [6]:
stacked_df_cols = ["open","high","low","close","vol","vol_coin","vwap","obv","change","direction"]
print(len(stacked_df_cols))
print(len(df)-1)
df.loc[0:len(df)-2,stacked_df_cols]

10
9


,open,high,low,close,vol,vol_coin,vwap,obv,change,direction
0,68065,68070.5,68052,68053.5,367534,5.39970161,0.185162,-367534.0,-11.5,-1
1,68006.5,68075,67990,68065,2317435,34.06543447,0.029371,1949901.0,58.5,1
2,68120,68135,68006.5,68006.5,2458651,36.12209538,0.027660,-508750.0,-113.5,-1
3,68186,68186,68107,68120,2779844,40.79392886,0.024505,-3288594.0,-66.0,-1
4,68197,68393.5,68181,68186,4760662,69.70852656,0.014323,-8049256.0,-11.0,-1
5,68002,68200,68001,68197,4181886,61.40196159,0.016308,-3867370.0,195.0,1
6,68067.5,68087.5,67947,68002,2924624,42.99343437,0.023252,-6791994.0,-65.5,-1
7,68011.5,68140,68006,68067.5,2634776,38.70310042,0.025834,-4157218.0,56.0,1
8,67988,68024.5,67966.5,68011.5,2526190,37.14812638,0.026923,-1631028.0,23.5,1


In [7]:
len(prep_df.columns)

97

In [8]:
prep_df.head()

,time,day,hour,10MA,10STD,bb_upper_band,bb_lower_band,open_0,high_0,low_0,...,open_8,high_8,low_8,close_8,vol_8,vol_coin_8,vwap_8,obv_8,change_8,direction_8
0,2024-05-30 05:30:00,30,5,68069.7,75.417431,68220.534862,67918.865138,68065,68070.5,68052,...,67988,68024.5,67966.5,68011.5,2526190,37.14812638,0.026923,-1631028.0,23.5,1


# Creating a database

In [ ]:
from sqlalchemy import create_engine
#engine = create_engine("sqlite+pysqlite:///:memory:", echo=True)

engine = create_engine('sqlite:///BTCUSDTstream-test.db')

In [ ]:
# write function to transform data from bybit call    

In [ ]:
prep_df.to_sql('BTCUSDT',engine, if_exists = 'append', index=False)

In [ ]:
pd.read_sql('BTCUSDT',engine)

In essence the function calls would look like this: 

response = fetch_kline()
pred_df = prepare_data(response)

entry,stop_loss,take_profit = make_prediction(prep_df) # add column if trade or not 

write_db(prep_df)

place_order(entry,stop_loss,take_profit)

